# Trash code

### Betweeness centrality

### Betweeness Centrality T_cells and B_Cells
Betweenness centrality of a node 𝑣 is the sum of the fraction of all-pairs shortest paths that pass through 𝑣

$c_{B}(v)=\sum_{s, t \in V} \frac{\sigma(s, t \mid v)}{\sigma(s, t)}$

In [ ]:
#def Intersection(lst1, lst2):
#    return set(lst1).intersection(lst2)

#subgraphs = [cell_graph.subgraph(c).copy() for c in components]
#n =len(subgraphs)
#largest_component = subgraphs[n-1]
#nodes_in_largest = largest_component.nodes()
#t_b_nk_cells = list(nodes[nodes['phenotype'].apply(lambda phenotype: phenotype in ['T'])].cell_id.values)
#tumor_cells = list(nodes[nodes['phenotype'].apply(lambda phenotype: phenotype in ['tumor'])].cell_id.values)

#t_b_nk_cells = Intersection(nodes_in_largest,t_b_nk_cells)
#tumor_cells = Intersection(nodes_in_largest,tumor_cells)

#bc = nx.betweenness_centrality_subset(largest_component, t_b_nk_cells ,tumor_cells)
#cell_bc_df = pd.DataFrame(bc.items(),columns=['cell_id','betweeness_centrality'])

#len(cell_bc_df)

#cell_bc_df.describe()

#nodes = nodes.merge(cell_bc_df, on ='cell_id', how='left')

# Analysis on component features

### Components dataset

In [ ]:
directory = './data/output/'
output_directory = os.listdir(directory)
feature_inspection = pd.DataFrame({})
select_features = ['component','AUC_area']

for folder in output_directory[:]:
    if folder.startswith('DEEPMEL'):
        name = folder.split('_')[1]
        component_features = pd.read_csv(directory + folder +'/component_features.csv')
        component_features = component_features[select_features]
        component_features_pivotted = pd.pivot_table(component_features, values = select_features[1], columns = select_features[0])
        component_features_pivotted.columns.name = None
        component_features_pivotted = component_features_pivotted.reset_index()
        component_features_pivotted = component_features_pivotted.drop(['index'], axis =1)
        m = len(component_features_pivotted.columns)
        columns = component_features_pivotted.columns
        new_names = ['component_'+str(i) for i in range(m)]
        renaming ={}
        for old_name, new_name in zip(columns,new_names):
            renaming[old_name] = new_name
        
        component_features_pivotted = component_features_pivotted.rename(columns = renaming)    

        component_features_pivotted['name'] = name
        
        feature_inspection = pd.concat([feature_inspection,component_features_pivotted])
        
        feature_inspection = feature_inspection.fillna(0)

In [ ]:
feature_inspection['component_600'].value_counts()

In [ ]:
n = len(feature_inspection)
for column in feature_inspection.columns:
    if (feature_inspection[column].value_counts()[0])/n > 0.92 :
        feature_inspection = feature_inspection.drop([column], axis=1 )
        
    

In [ ]:
len(feature_inspection.columns)

In [ ]:
feature_inspection['label'] = feature_inspection.name.apply(lambda x: labeling(x))

In [ ]:
pca = PCA(n_components= 10)
label = feature_inspection.label.values
features_scaled= feature_inspection.drop(['name','label'],axis=1)
principalComponents = pca.fit_transform(features_scaled)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10'])
principalDf['label'] = label

In [ ]:
sns.pairplot(principalDf, hue='label', corner =True).savefig(PLOT_DIRECTORY+select_features[1]+'_PCA.png')
plt.close()

In [ ]:
correlation_nodes = principalDf.corr()
f, ax = plt.subplots(figsize=(10, 10))
ax = sns.heatmap(correlation_nodes.iloc[:,10:],annot=True)
plt.savefig(PLOT_DIRECTORY + select_features[1]+'_correlation_label.png')
plt.close()

In [ ]:
label = principalDf.label.values
nodes_columns = principalDf.columns[:]
spearmanr_corr ={}


for column in nodes_columns:
    if column != 'name' :
        column_value = principalDf[column].values
        spearmanr_corr[column] = scipy.stats.spearmanr(column_value, label).correlation

    
spearmanr_corr = pd.DataFrame(spearmanr_corr, index = [0])
spearmanr_corr = pd.melt(spearmanr_corr).set_index(['variable'])
f, ax = plt.subplots(figsize=(10, 20))
ax = sns.heatmap(spearmanr_corr,annot=True)
plt.savefig(PLOT_DIRECTORY + select_features[1]+'_spearman_rho.png')
plt.close()

In [ ]:
label = principalDf.label.values
laplace_columns_ = principalDf.columns[:]
kendall_corr ={}
for column in laplace_columns_:
    if column != 'name' :
        column_value = principalDf[column].values
        kendall_corr[column] = scipy.stats.kendalltau(column_value, label).correlation

kendall_corr = pd.DataFrame(kendall_corr, index = [0])
kendall_corr = pd.melt(kendall_corr).set_index(['variable'])
f, ax = plt.subplots(figsize=(10, 20))
ax = sns.heatmap(kendall_corr,annot=True)
plt.savefig(PLOT_DIRECTORY +  select_features[1]+'_kendall_tau.png')
plt.close()